In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matrixprofile.discover import snippets
import pandas as pd
from pyts.image import RecurrencePlot
import tensorflow as tf
import json
from sklearn.model_selection import train_test_split

In [ ]:
def from_np_array(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.array(ast.literal_eval(array_string))

In [ ]:
p = Path('../Dataset/WalkRun1_80_3800_200.csv')

df = pd.read_csv(p,converters={"neighbors":json.loads})

In [ ]:
 np.fromstring

In [333]:

rp = RecurrencePlot(threshold='point')
X_rp = rp.fit_transform(np.array(df.neighbors.iloc[0][0]).reshape(1,-1))
X=[]
y=[]
for i,item in df.iterrows():
    for j in item.neighbors:
        if rp.transform(np.array(j).reshape(1,-1))[0].shape == (300,300):
            X.append(rp.transform(np.array(j).reshape(1,-1))[0])
            y.append(item["key"])
X=np.stack(np.array(X))
y=np.array(y)

In [332]:
y

array([[1.]], dtype=float32)

In [330]:
y = tf.keras.utils.to_categorical(y)

In [331]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

ValueError: With n_samples=1, test_size=0.33 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
y_test

In [ ]:
from keras.layers import TimeDistributed
from keras.layers import Reshape
from keras.layers import Conv2D,Lambda, Activation, LSTM
from keras.utils import to_categorical
from keras.models import Model,Input
import keras 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Reshape,GlobalAveragePooling2D
from keras import backend as K
from tensorflow.keras.layers import GRU

In [ ]:
X_train.shape

In [325]:
def model():
    model = Sequential()
    input_layer = Input((X_train.shape[2], X_train.shape[2], 1),name="img_input",dtype='float32')
    output = Conv2D(64, (5, 5), kernel_initializer='he_normal',padding='same',  activation='relu')(input_layer)
    output = MaxPooling2D(pool_size=(2, 2))(output)
    output = Conv2D(64, (5, 5), kernel_initializer='he_normal',padding='same',  activation='relu')(output)
    output = MaxPooling2D(pool_size=(2, 2))(output)
    output = Conv2D(64, (5, 5), kernel_initializer='he_normal', padding='same', activation='relu')(output)
    output = GlobalAveragePooling2D()(output)
    output = Dropout(0.25)(output)
    output = Dense(y.shape[1])(output)
    y_pred = Activation('softmax', name='softmax')(output)
    
    
    model = Model(inputs=input_layer, outputs=y_pred)
    model.summary() 
    model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=[tf.keras.metrics.Accuracy()])
    
    return model

model = model()

Model: "model_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img_input (InputLayer)       [(None, 50, 50, 1)]       0         
_________________________________________________________________
conv2d_104 (Conv2D)          (None, 50, 50, 64)        1664      
_________________________________________________________________
max_pooling2d_81 (MaxPooling (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_105 (Conv2D)          (None, 25, 25, 64)        102464    
_________________________________________________________________
max_pooling2d_82 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_106 (Conv2D)          (None, 12, 12, 64)        102464    
_________________________________________________________________
global_average_pooling2d_19  (None, 64)                0  

In [310]:
histor = model.fit(X_train,
          y_train,
          validation_data=(X_valid,y_valid),
          #callbacks=[learning_rate_reduction], 
          batch_size=25, epochs=200)

Epoch 1/200
187/187 [==============================] - 45s 236ms/step - loss: 2.1777 - accuracy: 0.0000e+00 - val_loss: 1.5930 - val_accuracy: 0.0000e+00
Epoch 2/200
187/187 [==============================] - 41s 219ms/step - loss: 1.5898 - accuracy: 0.0000e+00 - val_loss: 1.4146 - val_accuracy: 0.0000e+00
Epoch 3/200
187/187 [==============================] - 42s 225ms/step - loss: 1.4503 - accuracy: 0.0000e+00 - val_loss: 1.3227 - val_accuracy: 0.0000e+00
Epoch 4/200
187/187 [==============================] - 43s 231ms/step - loss: 1.3513 - accuracy: 0.0000e+00 - val_loss: 1.2507 - val_accuracy: 0.0000e+00
Epoch 5/200
187/187 [==============================] - 41s 221ms/step - loss: 1.2748 - accuracy: 0.0000e+00 - val_loss: 1.1818 - val_accuracy: 0.0000e+00
Epoch 6/200
187/187 [==============================] - 42s 227ms/step - loss: 1.1810 - accuracy: 0.0000e+00 - val_loss: 1.1123 - val_accuracy: 0.0000e+00
Epoch 7/200
187/187 [==============================] - 41s 221ms/step - loss

KeyboardInterrupt: 

In [311]:
pred= model.predict(X_test)
np.unique(np.argmax(y_valid,axis=-1),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([139, 325, 309, 223, 303, 326, 203, 164, 215,  88], dtype=int64))

In [312]:
np.unique(np.argmax(y_train,axis=-1),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([247, 647, 618, 460, 577, 739, 427, 318, 466, 159], dtype=int64))

In [313]:
np.argmax(pred,axis=-1)
y_test_arg = np.argmax(y_test,axis=-1)
j = 0
for i, item in enumerate(np.argmax(pred,axis=-1)):
    if item!=y_test_arg[i]:
        j+=1
print((len(pred)-j)/len(pred))    

0.8697167755991285
